### Extraindo informações mais gerais dos produtos na categoria Ofertas:
* Título
* Marca
* Preço com Desconto
* Preço sem desconto

In [7]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup


chrome_options = Options()
chrome_options.add_argument("--headless") 


urls = [
    
    "https://drogariasoares.com.br/kitmix-ofertas?pagina=3"
]


all_products_data = []


for url in urls:
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(3)  

    
    html_content = driver.page_source

   
    driver.quit()

    
    soup = BeautifulSoup(html_content, "html.parser")

    
    produtos = soup.find_all("div", class_="item-prod")

    
    for produto in produtos:
        titulo = produto.find("h2").text.strip()
        marca = produto.find("small", class_="marca").text.strip()
        preco_sem_desconto_element = produto.find("li", class_="preco text-center").find("p", class_='preco-de')
        preco_sem_desconto = preco_sem_desconto_element.text.strip() if preco_sem_desconto_element else ""
        preco_com_desconto = produto.find("p", class_="preco-por").strong.span.text.strip()

       
        product_data = {
            "Título": titulo,
            "Marca": marca,
            "Preço sem Desconto": preco_sem_desconto,
            "Preço com Desconto": preco_com_desconto
        }
        all_products_data.append(product_data)


df1 = pd.DataFrame(all_products_data)


df1.drop_duplicates(inplace=True)

In [5]:
df1

,Título,Marca,Preço sem Desconto,Preço com Desconto
0,DORFLEX COM 24 COMPRIMIDOS,SANOFI CONSUMO,,"R$ 16,99"
1,TARGIFOR C 1G16 COMPRIMIDOS,SANOFI CONSUMO,,"R$ 43,49"
2,CREME SERUM FACIAL NIVEA CELLULAR LUMINOUS NOI...,NIVEA,"R$ 135,90","R$ 99,90"
3,CENTRUM MULHER COM 60 COMPRIMIDOS,HALEON,"R$ 132,89","R$ 99,90"
4,PROTETOR SOLAR FACIAL ANTHELIOS FPS80 AIRLICIU...,LOREAL BR- DIV DCA LA ROCHE,"R$ 98,90","R$ 78,90"
...,...,...,...,...
62,"ESMALTE RISQUE DIAMOND GEL VERMELHO RUBI 9,5ML",COTY BRASIL -RISQUE,"R$ 11,90","R$ 8,99"
63,CARMED LABIAL BFF ROSA GLITTER EFEITO GLOSS 10G,CIMED,"R$ 24,99","R$ 18,99"
64,APARELHO DE DEPILAÇAO GILLETTE FEMININO VENUS ...,GILLETTE,"R$ 22,90","R$ 17,99"
65,NEUTROGENA GEL DE LIMPEZA FACIAL DEEP CLEAN GR...,JOHNSON & JOHNSON-HIGIENE,"R$ 42,20","R$ 36,90"


### Extraindo os códigos dos produtos na categoria Ofertas:
* ID
* SKU
##### OBS: Infelizmente, no site da Drogaria Soares não foi localizado o código relativo ao EAN, por esse motivo, o mesmo não foi apresentado nesse código.

In [3]:
import requests
from pyquery import PyQuery as pq
import re
import concurrent.futures
import pandas as pd

def extrair_info_produto(url):
    response = requests.get(url)
    doc = pq(response.text)
    
    # Encontrar o script que contém as informações do produto
    script_content = doc('script:contains("id:")').text()
    
    # Procurar o ID e SKU do produto no conteúdo do script
    id_match = re.search(r"id:'(\d+)'", script_content)
    sku_match = re.search(r"sku:'(\d+)'", script_content)
    
    if id_match and sku_match:
        id_produto = id_match.group(1)
        sku_produto = sku_match.group(1)
        return id_produto, sku_produto
    else:
        return None, None

def extrair_info_produto_wrapper(url):
    try:
        return extrair_info_produto(url)
    except Exception as e:
        print(f"Erro ao extrair informações de {url}: {e}")
        return None, None

def extrair_links_produtos():
    urls = [
        
        "https://drogariasoares.com.br/kitmix-ofertas?pagina=3"  
        
    ]
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
    }
    links = []

    for url in urls:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            doc = pq(response.text)
            products = doc('.link-prod')

            for product in products:
                href = pq(product).attr('href')
                if 'a-4-body-lotion-maca-220g' not in href:
                    links.append(href)

    return links

links_produtos = extrair_links_produtos()

dados_produtos = []
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    resultados = executor.map(extrair_info_produto_wrapper, ['https://drogariasoares.com.br' + link for link in links_produtos])
    for resultado in resultados:
        if resultado[0] is not None and resultado[1] is not None:
            dados_produtos.append(resultado)

df2 = pd.DataFrame(dados_produtos, columns=['ID do Produto', 'SKU'])
df2.drop_duplicates(inplace=True)


C:\Users\lalaz\AppData\Local\Temp\ipykernel_20096\4166853924.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Unificando e Tratando os Dataframes:

In [5]:
reindexed_df2 = df2.set_index(pd.Index(range(67)))

In [8]:

df1["ID"] = reindexed_df2["ID do Produto"]
df1["SKU"] = reindexed_df2["SKU"]


In [9]:
df1

,Título,Marca,Preço sem Desconto,Preço com Desconto,ID,SKU
0,DORFLEX COM 24 COMPRIMIDOS,SANOFI CONSUMO,,"R$ 16,99",370804,97531
1,TARGIFOR C 1G16 COMPRIMIDOS,SANOFI CONSUMO,,"R$ 43,49",378739,89375
2,CREME SERUM FACIAL NIVEA CELLULAR LUMINOUS NOI...,NIVEA,"R$ 135,90","R$ 99,90",9991507,104199
3,CENTRUM MULHER COM 60 COMPRIMIDOS,HALEON,"R$ 132,89","R$ 99,90",618505,68464
4,PROTETOR SOLAR FACIAL ANTHELIOS FPS80 AIRLICIU...,LOREAL BR- DIV DCA LA ROCHE,"R$ 98,90","R$ 78,90",9995799,108427
...,...,...,...,...,...,...
62,"ESMALTE RISQUE DIAMOND GEL VERMELHO RUBI 9,5ML",COTY BRASIL -RISQUE,"R$ 11,90","R$ 8,99",9991050,103733
63,CARMED LABIAL BFF ROSA GLITTER EFEITO GLOSS 10G,CIMED,"R$ 24,99","R$ 18,99",9995495,107135
64,APARELHO DE DEPILAÇAO GILLETTE FEMININO VENUS ...,GILLETTE,"R$ 22,90","R$ 17,99",9994756,107065
65,NEUTROGENA GEL DE LIMPEZA FACIAL DEEP CLEAN GR...,JOHNSON & JOHNSON-HIGIENE,"R$ 42,20","R$ 36,90",3309909,95957


In [10]:
df1.to_csv('Ofertas_Soares.csv')